# Proyecto Ecobici
1. Obener información de Programa Ecobici
2. Limpiar la base de datos. Estructura homogénea. - Script Artur
3. Missing Values, NA's y Nulos - Análisis
4. Ubicación de cicloestaciones - visualización geográfica
5. EDA - Estadísticas(), Insights y Visualizaciones (histogramas duración de viajes, agrupaciones+agregaciones)
6. Algoritmo de Clustering para identificar ubicación de hotspots
7. Recomendaciones de Redes - dos enfoques\
    a) Content Based Algo - Comparar necesidades de usuarios con características de las cicloestaciones. Entrenar un modelo con el producto punto como medida de similaridad.\
    b) Recomendaciones con Collaborative Filtering
8. Conclusiones, Propuesta de Nuevos Modelos

9. Agrupar por cicloestación o por usuario

10. Algoritmos aglomerativos

## 1. Obener información del Programa Ecobici

In [1]:
import io
import pandas as pd

In [2]:
# función para leer datos del URL de ecobici
def leer_historico_ecobici(anio, mes):
  import requests
  csv_url = f"https://www.ecobici.cdmx.gob.mx/sites/default/files/data/usages/{anio}-{mes}.csv"
  req = requests.get(csv_url)
  url_content = req.content
  rawData = pd.read_csv(io.StringIO(url_content.decode('utf-8')))
  return rawData

In [7]:
"""""

# periodos a consultar
consultas = [
  #(2022, "06"), # info de Junio aún no está completa para consultar
  (2022, "05"),
  (2022, "04"),
  (2022, "03"),
  (2022, "02"),
  (2022, "01"), 
  (2021, "12"),
  (2021, "11"),
  (2021, "10"),
  (2021, "09"),
  (2021, "08"),
  (2021, "07"),
  (2021, "06"),
  (2021, "05"),
  (2021, "04"),
  (2021, "03"),
  (2021, "02"),
  (2021, "01"), 
  #(2020, "12"), # detalle a atender -- revisar info de dic/2020 en especifico
  (2020, "11"),
  (2020, "10"),
  (2020, "09"),
  (2020, "08"),
  (2020, "07"),
  (2020, "06"),
  (2020, "05"),
  (2020, "04"),
  (2020, "03"),
  (2020, "02"),
  (2020, "01"),
  (2019, "12"),
  (2019, "11"),
  (2019, "10"),
  (2019, "09"),
  (2019, "08"),
  (2019, "07"),
  (2019, "06"),
  (2019, "05"),
  (2019, "04"),
  (2019, "03"),
  (2019, "02"),
  (2019, "01")
]

"""

In [18]:
consultas = [
  #(2020, "12"),
  (2020, "11"),
  (2020, "10"),
  (2020, "09"),
  (2020, "08"),
  (2020, "07"),
  (2020, "06"),
  (2020, "05"),
  (2020, "04"),
  (2020, "03"),
  (2020, "02"),
  (2020, "01")]

In [19]:
# leer historico para cada mes especificado en lista consultas
consulta_df = []
for consulta in consultas:
  print(consulta)
  consulta_df.append(leer_historico_ecobici(*consulta))

(2020, '11')
(2020, '10')
(2020, '09')
(2020, '08')
(2020, '07')
(2020, '06')
(2020, '05')
(2020, '04')
(2020, '03')


C:\Users\mike_\AppData\Local\Temp\ipykernel_6184\2215206597.py:6: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  rawData = pd.read_csv(io.StringIO(url_content.decode('utf-8')))


(2020, '02')
(2020, '01')


In [20]:
# consolidar la información en un pandas dataset
df = pd.DataFrame({})
for df_temp in consulta_df:
  df = df.append(df_temp)

C:\Users\mike_\AppData\Local\Temp\ipykernel_6184\1202030006.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\mike_\AppData\Local\Temp\ipykernel_6184\1202030006.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\mike_\AppData\Local\Temp\ipykernel_6184\1202030006.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\mike_\AppData\Local\Temp\ipykernel_6184\1202030006.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(df_temp)
C:\Users\mike_\AppData\Local\Temp\ipykernel_6184\1202030006.py:4: FutureWarning: The frame.append me

In [21]:
# dimension de filas y columnas
df.shape 

(3774961, 10)

In [22]:
df.head()

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,BikeID
0,M,48.0,9989.0,140.0,31/10/2020,23:46:40,85,01/11/2020,0:03:35,NaN
1,M,29.0,6898.0,276.0,01/11/2020,5:16:20,424,01/11/2020,5:38:13,NaN
2,F,43.0,11061.0,60.0,01/11/2020,8:50:57,232,01/11/2020,9:27:09,NaN
3,F,35.0,11097.0,408.0,01/11/2020,8:25:52,96,01/11/2020,9:28:59,NaN
4,F,27.0,10308.0,379.0,01/11/2020,9:35:55,394,01/11/2020,9:38:28,NaN


In [23]:
df.sample(7)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,BikeID
24453,M,22.0,10024.0,271.0,03/10/2020,8:44:42,23,03/10/2020,8:56:43,NaN
169154,M,53.0,11470.0,141.0,20/10/2020,11:18:30,141,20/10/2020,11:45:52,NaN
174337,M,28.0,9986,113.0,10/01/2020,16:12:05,270,10/01/2020,16:19:20,NaN
48736,M,38.0,7354,20.0,03/03/2020,9:30:14,14,03/03/2020,9:42:02,NaN
268596,F,25.0,7465,64.0,15/01/2020,7:52:38,36,15/01/2020,7:59:48,NaN
565573,F,71.0,11730,5.0,28/01/2020,8:35:29,56,28/01/2020,8:50:32,NaN
135807,F,32.0,1583.0,295.0,18/11/2020,9:26:53,74,18/11/2020,9:45:14,NaN


In [25]:
# Variables númericas y variables categoricas
num_vars = df.columns[df.dtypes != 'object']
cat_vars = df.columns[df.dtypes == 'object']

In [27]:
# análisis de missing values en variables numéricas
df[num_vars].isnull().sum().sort_values(ascending=False)

BikeID                   3508023
Edad_Usuario                  32
Ciclo_Estacion_Retiro         32
Ciclo_Estacion_Arribo          0
dtype: int64

In [28]:
# análisis de missing values en variables numéricas en porcentaje
df[num_vars].isnull().sum().sort_values(ascending=False)/len(df)  # id de bicicleta muchos missing

BikeID                   0.929287
Edad_Usuario             0.000008
Ciclo_Estacion_Retiro    0.000008
Ciclo_Estacion_Arribo    0.000000
dtype: float64

In [29]:
# estadisticas rapidas
df.describe()

,Edad_Usuario,Ciclo_Estacion_Retiro,Ciclo_Estacion_Arribo,BikeID
count,3.774929e+06,3.774929e+06,3.774961e+06,266938.000000
mean,3.609266e+01,1.863721e+02,1.850312e+02,9916.554421
std,1.072206e+01,1.337225e+02,1.331043e+02,2684.602780
min,1.600000e+01,1.000000e+00,1.000000e+00,809.000000
25%,2.800000e+01,6.900000e+01,6.900000e+01,8233.000000
50%,3.300000e+01,1.600000e+02,1.580000e+02,10078.000000
75%,4.200000e+01,2.860000e+02,2.820000e+02,12049.000000
max,1.200000e+02,3.002000e+03,3.003000e+03,14207.000000


In [30]:
# tipos de datos
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3774961 entries, 0 to 676907
Data columns (total 10 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Genero_Usuario         object 
 1   Edad_Usuario           float64
 2   Bici                   object 
 3   Ciclo_Estacion_Retiro  float64
 4   Fecha_Retiro           object 
 5   Hora_Retiro            object 
 6   Ciclo_Estacion_Arribo  int64  
 7   Fecha_Arribo           object 
 8   Hora_Arribo            object 
 9   BikeID                 float64
dtypes: float64(3), int64(1), object(6)
memory usage: 316.8+ MB


In [ ]:
# cambiar tipos de datos de fecha # crear dos columnas: datetome inicio y datetime término

### Cicloestaciones

In [31]:
estaciones = pd.read_csv('https://raw.githubusercontent.com/gonzalezgouveia/proyecto-ecobici/master/datos/ecobiciestaciones.csv')

In [32]:
estaciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268 entries, 0 to 267
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          268 non-null    int64  
 1   principal   268 non-null    object 
 2   secundario  268 non-null    object 
 3   referencia  178 non-null    object 
 4   colonia     268 non-null    object 
 5   delegacion  268 non-null    object 
 6   longitud    268 non-null    float64
 7   latitud     268 non-null    float64
 8   nombre      268 non-null    object 
dtypes: float64(2), int64(1), object(6)
memory usage: 19.0+ KB


In [33]:
estaciones.describe()

,id,longitud,latitud
count,268.000000,268.000000,268.000000
mean,135.895522,-99.169222,19.423328
std,78.647154,0.018256,0.015289
min,1.000000,-99.207808,19.249080
25%,67.750000,-99.180484,19.414257
50%,136.500000,-99.168895,19.426420
75%,203.250000,-99.158116,19.432689
max,275.000000,-99.104160,19.444272


In [34]:
estaciones.sample(7)

,id,principal,secundario,referencia,colonia,delegacion,longitud,latitud,nombre
210,213,Av. Horacio,FFCC Cuernavaca,Frente al No. 1550,Polanco,Miguel Hidalgo,-99.206967,19.435197,Av. Horacio-FFCC Cuernavaca
112,115,Roma,Dinamarca,A un costado de Mac´Donals,Juárez,Cuauhtémoc,-99.159698,19.429947,Roma-Dinamarca
253,256,Gómez Farías,Sadi Carnot,Frente a la UVM,San Rafael,Cuauhtémoc,-99.158313,19.437167,Gómez Farías-Sadi Carnot
31,32,Londres,Sevilla,NaN,Juárez,Cuauhtémoc,-99.169700,19.422850,Londres-Sevilla
65,66,Mazatlán,Francisco Márquez,NaN,Condesa,Cuauhtémoc,-99.178480,19.411810,Mazatlán-Francisco Márquez
56,57,Cuernavaca,Juan Escutia,NaN,Condesa,Cuauhtémoc,-99.174770,19.414970,Cuernavaca-Juan Escutia
129,132,Colima,Frontera,Frente a Edificio Marisol,Roma Norte,Cuauhtémoc,-99.156450,19.420667,Colima-Frontera


### Analizando los csv mensuales 

In [1]:
import pandas as pd

In [49]:
# Reading year 2019
jan_2019 = pd.read_csv('data/2019-01.csv')
feb_2019 = pd.read_csv('data/2019-02.csv')
mar_2019 = pd.read_csv('data/2019-03.csv')
apr_2019 = pd.read_csv('data/2019-04.csv')
may_2019 = pd.read_csv('data/2019-05.csv')
jun_2019 = pd.read_csv('data/2019-06.csv')
jul_2019 = pd.read_csv('data/2019-07.csv')
aug_2019 = pd.read_csv('data/2019-08.csv')
sep_2019 = pd.read_csv('data/2019-09.csv')
oct_2019 = pd.read_csv('data/2019-10.csv')
nov_2019 = pd.read_csv('data/2019-11.csv')
dec_2019 = pd.read_csv('data/2019-12.csv')

C:\Users\mike_\AppData\Local\Temp\ipykernel_8008\1916318915.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  jan_2019 = pd.read_csv('data/2019-01.csv')
C:\Users\mike_\AppData\Local\Temp\ipykernel_8008\1916318915.py:3: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  feb_2019 = pd.read_csv('data/2019-02.csv')
C:\Users\mike_\AppData\Local\Temp\ipykernel_8008\1916318915.py:4: DtypeWarning: Columns (2,6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  mar_2019 = pd.read_csv('data/2019-03.csv')
C:\Users\mike_\AppData\Local\Temp\ipykernel_8008\1916318915.py:11: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  oct_2019 = pd.read_csv('data/2019-10.csv')
C:\Users\mike_\AppData\Local\Temp\ipykernel_8008\1916318915.py:12: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set lo

In [50]:
jan_2019.columns == feb_2019.columns # january columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [51]:
jan_2019.shape

(697289, 9)

In [52]:
jan_2019.columns

Index(['Genero_Usuario', 'Edad_Usuario', 'Bici', 'Ciclo_Estacion_Retiro',
       'Fecha_Retiro', 'Hora_Retiro', 'Ciclo_Estacion_Arribo', 'Fecha_Arribo',
       'Hora_Arribo'],
      dtype='object')

In [53]:
mar_2019.columns # march-2019 got an extra column

Index(['Genero_Usuario', 'Edad_Usuario', 'Bici', 'Ciclo_Estacion_Retiro',
       'Fecha_Retiro', 'Hora_Retiro', 'Ciclo_Estacion_Arribo', 'Fecha_Arribo',
       'Hora_Arribo', 'Unnamed: 9'],
      dtype='object')

In [54]:
mar_2019.shape

(752299, 10)

In [55]:
mar_2019.sample(7)

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Unnamed: 9
603805,M,23,7262,384,26/03/2019,08:32:08,166,26/03/2019,08:46:06,NaN
636408,M,28,12217,266,27/03/2019,08:38:01,23,27/03/2019,09:01:12,NaN
53498,M,35,10463,15,03/03/2019,19:39:03,46,03/03/2019,19:47:28,NaN
421185,M,27,3282,169,19/03/2019,06:59:25,182,19/03/2019,07:01:07,NaN
84356,M,31,7262,347,05/03/2019,07:57:39,390,05/03/2019,08:03:28,NaN
345900,M,31,9123,51,14/03/2019,17:12:12,32,14/03/2019,17:16:46,NaN
127375,F,30,11826,59,06/03/2019,16:11:39,163,06/03/2019,16:23:03,NaN


In [56]:
mar_2019[mar_2019['Unnamed: 9']=='08:41:11'] # row with incorrectly assigned value at 'Hora_Arribo'

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Unnamed: 9
113432,M,44,9944,267,06/03/2019,08:06:23,32:11.8,10,00:00:00,08:41:11


In [57]:
# In March 2019, 'Hora_Arribo' column was not correctly registered. Assign it to its corresponging column
mar_2019.at[113432, 'Hora_Arribo'] = '08:41:11'

In [58]:
# check the index '113432' 
mar_2019.iloc[[113432]]

,Genero_Usuario,Edad_Usuario,Bici,Ciclo_Estacion_Retiro,Fecha_Retiro,Hora_Retiro,Ciclo_Estacion_Arribo,Fecha_Arribo,Hora_Arribo,Unnamed: 9
113432,M,44,9944,267,06/03/2019,08:06:23,32:11.8,10,08:41:11,08:41:11


In [59]:
# now we can drop the whole column 'Unnamed: 9'
mar_2019.drop('Unnamed: 9', inplace=True, axis=1)

In [60]:
# check march 2019 structure
mar_2019.shape

(752299, 9)

In [61]:
# verify mar_2019 matches the structure of the other months
mar_2019.columns == may_2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [62]:
apr_2019.columns == jan_2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [30]:
jan_2019.columns == may_2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [31]:
may_2019.columns == jun_2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [32]:
jun_2019.columns == jul_2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [33]:
jul_2019.columns == aug_2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [34]:
aug_2019.columns == sep_2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [35]:
sep_2019.columns == oct_2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [36]:
oct_2019.columns == nov_2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [37]:
nov_2019.columns == dec_2019.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True])

In [63]:
# Proceed to merge all 2019 months into a single dataframe
months_2019 = [jan_2019, feb_2019, mar_2019, apr_2019, may_2019, jun_2019, jul_2019, aug_2019, sep_2019, oct_2019, nov_2019, dec_2019]
year_2019 = pd.concat(months_2019, ignore_index=True)

In [64]:
year_2019.shape

(7725442, 9)

In [82]:
sum(month.shape[0] for month in months_2019) == year_2019.shape[0] # Verify merged dataset dimension

True

In [83]:
# Reading year 2020
jan_2020 = pd.read_csv('data/2020-01.csv')
feb_2020 = pd.read_csv('data/2020-02.csv')
mar_2020 = pd.read_csv('data/2020-03.csv')
apr_2020 = pd.read_csv('data/2020-04.csv')
may_2020 = pd.read_csv('data/2020-05.csv')
jun_2020 = pd.read_csv('data/2020-06.csv')
jul_2020 = pd.read_csv('data/2020-07.csv')
aug_2020 = pd.read_csv('data/2020-08.csv')
sep_2020 = pd.read_csv('data/2020-09.csv')
oct_2020 = pd.read_csv('data/2020-10.csv')
nov_2020 = pd.read_csv('data/2020-11.csv')
dec_2020 = pd.read_csv('data/2020-12.csv')

C:\Users\mike_\AppData\Local\Temp\ipykernel_8008\1299327238.py:4: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  mar_2020 = pd.read_csv('data/2020-03.csv')


UnicodeDecodeError: 'utf-8' codec can't decode bytes in position 15-16: invalid continuation byte

In [85]:
jan_2020.shape
jan_2020.columns

Index(['Genero_Usuario', 'Edad_Usuario', 'Bici', 'Ciclo_Estacion_Retiro',
       'Fecha_Retiro', 'Hora_Retiro', 'Ciclo_Estacion_Arribo', 'Fecha_Arribo',
       'Hora_Arribo'],
      dtype='object')